In [2]:
# Inport dependencies
import sqlite3
import pandas as pd
import os
import requests
import time
import csv
from config import api_key

# Fetching Geocode Data  
- Collecting longitude and latitude of cities from customer "https://api.opencagedata.com"

In [9]:
def geocode_location(address, api_key):
    base_url = "https://api.opencagedata.com/geocode/v1/json"
    params = {
        'q': address,
        'key': api_key,
    }
    response = requests.get(base_url, params=params)
    
    if response.status_code == 200:
        print(f"Successful API connection for address: {address}")
        data = response.json()
        if data['results']:
            lat = data['results'][0]['geometry']['lat']
            lng = data['results'][0]['geometry']['lng']
            return lat, lng
        else:
            raise ValueError("No results found")
    else:
        print(f"Request failed with status code: {response.status_code}")
        raise ValueError(f"Error: {response.status_code}")

In [11]:
# Verify the file path
file_path = 'Resources/Customers.csv'

In [12]:
#  Read the customer data from the CSV file
try:
    customer_df = pd.read_csv(file_path, encoding='latin1')  # Adjust encoding if necessary
except FileNotFoundError:
    print(f"File not found: {file_path}")
except Exception as e:
    print(f"An error occurred: {e}")

In [13]:
# Filter unique cities
unique_cities = customer_df['City'].unique()

In [14]:
unique_cities

array(['WANDEARAH EAST', 'MOUNT BUDD', 'WINJALLOK', ..., 'Felton',
       'Pump Station Three', 'Lorton'], dtype=object)

In [15]:
# Initialize the dictionary
geocode_dict = {
    'City': [],
    'Latitude': [],
    'Longitude': []
}

total_cities = len(unique_cities)
for index, city in enumerate(unique_cities):
    try:
        lat, lng = geocode_location(city, api_key)
        geocode_dict['City'].append(city)
        geocode_dict['Latitude'].append(lat)
        geocode_dict['Longitude'].append(lng)
        print(f"Processed {index + 1}/{total_cities}: {city} - Success")
    except Exception as e:
        geocode_dict['City'].append(city)
        geocode_dict['Latitude'].append(None)
        geocode_dict['Longitude'].append(None)
        print(f"Processed {index + 1}/{total_cities}: {city} - Failed ({e})")


Successful API connection for address: WANDEARAH EAST
Processed 1/8258: WANDEARAH EAST - Success
Successful API connection for address: MOUNT BUDD
Processed 2/8258: MOUNT BUDD - Success
Successful API connection for address: WINJALLOK
Processed 3/8258: WINJALLOK - Success
Successful API connection for address: MIDDLE RIVER
Processed 4/8258: MIDDLE RIVER - Success
Successful API connection for address: TAWONGA SOUTH
Processed 5/8258: TAWONGA SOUTH - Success
Successful API connection for address: TEMPLERS
Processed 6/8258: TEMPLERS - Success
Successful API connection for address: JUBILEE POCKET
Processed 7/8258: JUBILEE POCKET - Success
Successful API connection for address: KULLOGUM
Processed 8/8258: KULLOGUM - Success
Successful API connection for address: EDITH RIVER
Processed 9/8258: EDITH RIVER - Success
Successful API connection for address: KOTTA
Processed 10/8258: KOTTA - Success
Successful API connection for address: GREEN LAKE
Processed 11/8258: GREEN LAKE - Success
Successful 

In [16]:
# Create a DataFrame from the dictionary
geocode_df = pd.DataFrame(geocode_dict)

# Data cleaning
## Customer table
- creating a dataframe by `customers_df` and `geocode_df`
- Drop rows with any null values
- Make the city names proper
- Reorder the columns
- Split the Name column into 'First_Name' and 'Last_Name', handling potential extra spaces
- Drop the original Name column
- Change the column names to lowercase with underscores and reorder
- Convert the 'birthday' column to datetime
- Save the updated dataframe to a new CSV file in Output folder

In [17]:
merged_df = customer_df.merge(geocode_df, on='City', how='left')

In [ ]:
# Merge with customer_df on the City column
merged_df = pd.merge(customer_df, geocode_df, on='City', how='left')

In [19]:
merged_df

,CustomerKey,Gender,Name,City,State Code,State,Zip Code,Country,Continent,Birthday,Latitude,Longitude
0,301,Female,Lilly Harding,WANDEARAH EAST,SA,South Australia,5523,Australia,Australia,7/3/1939,-33.398609,138.047863
1,325,Female,Madison Hull,MOUNT BUDD,WA,Western Australia,6522,Australia,Australia,9/27/1979,-29.180810,115.636896
2,554,Female,Claire Ferres,WINJALLOK,VIC,Victoria,3380,Australia,Australia,5/26/1947,-36.840260,143.191632
3,786,Male,Jai Poltpalingada,MIDDLE RIVER,SA,South Australia,5223,Australia,Australia,9/17/1957,48.434307,-96.164072
4,1042,Male,Aidan Pankhurst,TAWONGA SOUTH,VIC,Victoria,3698,Australia,Australia,11/19/1965,-36.762541,147.138785
...,...,...,...,...,...,...,...,...,...,...,...,...
15261,2099600,Female,Denisa Duková,Houston,TX,Texas,77017,United States,North America,3/25/1936,29.758938,-95.367697
15262,2099618,Male,Justin Solórzano,Mclean,VA,Virginia,22101,United States,North America,2/16/1992,40.463179,-88.819661
15263,2099758,Male,Svend Petrussen,Wilmington,NC,North Carolina,28405,United States,North America,11/9/1937,39.745947,-75.546589
15264,2099862,Female,Lorenza Rush,Riverside,CA,California,92501,United States,North America,10/12/1937,33.721999,-116.037247


In [26]:
len(merged_df)

15266

In [29]:
# Drop rows with any null values
cleaned_df = merged_df.dropna()

In [30]:
len(merged_df)

15266

In [31]:
# Define conditions for valid latitude and longitude based on the given countries
conditions = [
    (cleaned_df['Country'] == 'Australia') & (cleaned_df['Latitude'] < 0) & (cleaned_df['Longitude'] > 0),
    (cleaned_df['Country'] == 'United States') & (cleaned_df['Latitude'] > 0) & (cleaned_df['Longitude'] < 0),
    (cleaned_df['Country'] == 'Canada') & (cleaned_df['Latitude'] > 0) & (cleaned_df['Longitude'] < 0),
    (cleaned_df['Country'] == 'France') & (cleaned_df['Latitude'] > 0) & (cleaned_df['Longitude'] > 0),
    (cleaned_df['Country'] == 'Germany') & (cleaned_df['Latitude'] > 0) & (cleaned_df['Longitude'] > 0),
    (cleaned_df['Country'] == 'Italy') & (cleaned_df['Latitude'] > 0) & (cleaned_df['Longitude'] > 0),
    (cleaned_df['Country'] == 'Netherlands') & (cleaned_df['Latitude'] > 0) & (cleaned_df['Longitude'] > 0),
    (cleaned_df['Country'] == 'United Kingdom') & (cleaned_df['Latitude'] > 0) & (cleaned_df['Longitude'] < 0),
]

# Combine conditions with an OR operator
valid_conditions = conditions[0]
for condition in conditions[1:]:
    valid_conditions |= condition
    
    
# Filter the DataFrame based on valid conditions
cleaned_df = cleaned_df[valid_conditions]


In [32]:
len(merged_df)

15266

In [34]:
# Print the cleaned DataFrame
cleaned_df

,CustomerKey,Gender,Name,City,State Code,State,Zip Code,Country,Continent,Birthday,Latitude,Longitude
0,301,Female,Lilly Harding,WANDEARAH EAST,SA,South Australia,5523,Australia,Australia,7/3/1939,-33.398609,138.047863
1,325,Female,Madison Hull,MOUNT BUDD,WA,Western Australia,6522,Australia,Australia,9/27/1979,-29.180810,115.636896
2,554,Female,Claire Ferres,WINJALLOK,VIC,Victoria,3380,Australia,Australia,5/26/1947,-36.840260,143.191632
4,1042,Male,Aidan Pankhurst,TAWONGA SOUTH,VIC,Victoria,3698,Australia,Australia,11/19/1965,-36.762541,147.138785
5,1086,Male,Hayden Clegg,TEMPLERS,SA,South Australia,5371,Australia,Australia,1/20/1954,-34.468680,138.746913
...,...,...,...,...,...,...,...,...,...,...,...,...
15261,2099600,Female,Denisa Duková,Houston,TX,Texas,77017,United States,North America,3/25/1936,29.758938,-95.367697
15262,2099618,Male,Justin Solórzano,Mclean,VA,Virginia,22101,United States,North America,2/16/1992,40.463179,-88.819661
15263,2099758,Male,Svend Petrussen,Wilmington,NC,North Carolina,28405,United States,North America,11/9/1937,39.745947,-75.546589
15264,2099862,Female,Lorenza Rush,Riverside,CA,California,92501,United States,North America,10/12/1937,33.721999,-116.037247


In [35]:
# Make the city names proper
cleaned_df['City'] = cleaned_df['City'].str.title()

In [36]:
cleaned_df

,CustomerKey,Gender,Name,City,State Code,State,Zip Code,Country,Continent,Birthday,Latitude,Longitude
0,301,Female,Lilly Harding,Wandearah East,SA,South Australia,5523,Australia,Australia,7/3/1939,-33.398609,138.047863
1,325,Female,Madison Hull,Mount Budd,WA,Western Australia,6522,Australia,Australia,9/27/1979,-29.180810,115.636896
2,554,Female,Claire Ferres,Winjallok,VIC,Victoria,3380,Australia,Australia,5/26/1947,-36.840260,143.191632
4,1042,Male,Aidan Pankhurst,Tawonga South,VIC,Victoria,3698,Australia,Australia,11/19/1965,-36.762541,147.138785
5,1086,Male,Hayden Clegg,Templers,SA,South Australia,5371,Australia,Australia,1/20/1954,-34.468680,138.746913
...,...,...,...,...,...,...,...,...,...,...,...,...
15261,2099600,Female,Denisa Duková,Houston,TX,Texas,77017,United States,North America,3/25/1936,29.758938,-95.367697
15262,2099618,Male,Justin Solórzano,Mclean,VA,Virginia,22101,United States,North America,2/16/1992,40.463179,-88.819661
15263,2099758,Male,Svend Petrussen,Wilmington,NC,North Carolina,28405,United States,North America,11/9/1937,39.745947,-75.546589
15264,2099862,Female,Lorenza Rush,Riverside,CA,California,92501,United States,North America,10/12/1937,33.721999,-116.037247


In [ ]:
# Reorder the columns
cleaned_df = cleaned_df[['CustomerKey', 'Name', 'Gender', 'Birthday', 'City', 'State Code', 'State', 'Zip Code', 'Country', 'Continent', 'Latitude', 'Longitude']]

In [41]:
# Split the Name column into First_Name and Last_Name, handling potential extra spaces
name_split = cleaned_df['Name'].str.split(n=1, expand=True)
cleaned_df['First_Name'] = name_split[0]
cleaned_df['Last_Name'] = name_split[1]


In [42]:
cleaned_df

,CustomerKey,Gender,Name,City,State Code,State,Zip Code,Country,Continent,Birthday,Latitude,Longitude,First_Name,Last_Name
0,301,Female,Lilly Harding,Wandearah East,SA,South Australia,5523,Australia,Australia,7/3/1939,-33.398609,138.047863,Lilly,Harding
1,325,Female,Madison Hull,Mount Budd,WA,Western Australia,6522,Australia,Australia,9/27/1979,-29.180810,115.636896,Madison,Hull
2,554,Female,Claire Ferres,Winjallok,VIC,Victoria,3380,Australia,Australia,5/26/1947,-36.840260,143.191632,Claire,Ferres
4,1042,Male,Aidan Pankhurst,Tawonga South,VIC,Victoria,3698,Australia,Australia,11/19/1965,-36.762541,147.138785,Aidan,Pankhurst
5,1086,Male,Hayden Clegg,Templers,SA,South Australia,5371,Australia,Australia,1/20/1954,-34.468680,138.746913,Hayden,Clegg
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15261,2099600,Female,Denisa Duková,Houston,TX,Texas,77017,United States,North America,3/25/1936,29.758938,-95.367697,Denisa,Duková
15262,2099618,Male,Justin Solórzano,Mclean,VA,Virginia,22101,United States,North America,2/16/1992,40.463179,-88.819661,Justin,Solórzano
15263,2099758,Male,Svend Petrussen,Wilmington,NC,North Carolina,28405,United States,North America,11/9/1937,39.745947,-75.546589,Svend,Petrussen
15264,2099862,Female,Lorenza Rush,Riverside,CA,California,92501,United States,North America,10/12/1937,33.721999,-116.037247,Lorenza,Rush


In [44]:
# Drop the original Name column
cleaned_df = cleaned_df.drop(columns=['Name'])

KeyError: "['Name'] not found in axis"

In [45]:
cleaned_df

,CustomerKey,Gender,City,State Code,State,Zip Code,Country,Continent,Birthday,Latitude,Longitude,First_Name,Last_Name
0,301,Female,Wandearah East,SA,South Australia,5523,Australia,Australia,7/3/1939,-33.398609,138.047863,Lilly,Harding
1,325,Female,Mount Budd,WA,Western Australia,6522,Australia,Australia,9/27/1979,-29.180810,115.636896,Madison,Hull
2,554,Female,Winjallok,VIC,Victoria,3380,Australia,Australia,5/26/1947,-36.840260,143.191632,Claire,Ferres
4,1042,Male,Tawonga South,VIC,Victoria,3698,Australia,Australia,11/19/1965,-36.762541,147.138785,Aidan,Pankhurst
5,1086,Male,Templers,SA,South Australia,5371,Australia,Australia,1/20/1954,-34.468680,138.746913,Hayden,Clegg
...,...,...,...,...,...,...,...,...,...,...,...,...,...
15261,2099600,Female,Houston,TX,Texas,77017,United States,North America,3/25/1936,29.758938,-95.367697,Denisa,Duková
15262,2099618,Male,Mclean,VA,Virginia,22101,United States,North America,2/16/1992,40.463179,-88.819661,Justin,Solórzano
15263,2099758,Male,Wilmington,NC,North Carolina,28405,United States,North America,11/9/1937,39.745947,-75.546589,Svend,Petrussen
15264,2099862,Female,Riverside,CA,California,92501,United States,North America,10/12/1937,33.721999,-116.037247,Lorenza,Rush


In [47]:
# Reorder the columns
cleaned_df = cleaned_df[['CustomerKey', 'First_Name', 'Last_Name', 'Gender', 'Birthday', 'City', 'State Code', 'State', 'Zip Code', 'Country', 'Continent', 'Latitude', 'Longitude']]

In [48]:
cleaned_df

,CustomerKey,First_Name,Last_Name,Gender,Birthday,City,State Code,State,Zip Code,Country,Continent,Latitude,Longitude
0,301,Lilly,Harding,Female,7/3/1939,Wandearah East,SA,South Australia,5523,Australia,Australia,-33.398609,138.047863
1,325,Madison,Hull,Female,9/27/1979,Mount Budd,WA,Western Australia,6522,Australia,Australia,-29.180810,115.636896
2,554,Claire,Ferres,Female,5/26/1947,Winjallok,VIC,Victoria,3380,Australia,Australia,-36.840260,143.191632
4,1042,Aidan,Pankhurst,Male,11/19/1965,Tawonga South,VIC,Victoria,3698,Australia,Australia,-36.762541,147.138785
5,1086,Hayden,Clegg,Male,1/20/1954,Templers,SA,South Australia,5371,Australia,Australia,-34.468680,138.746913
...,...,...,...,...,...,...,...,...,...,...,...,...,...
15261,2099600,Denisa,Duková,Female,3/25/1936,Houston,TX,Texas,77017,United States,North America,29.758938,-95.367697
15262,2099618,Justin,Solórzano,Male,2/16/1992,Mclean,VA,Virginia,22101,United States,North America,40.463179,-88.819661
15263,2099758,Svend,Petrussen,Male,11/9/1937,Wilmington,NC,North Carolina,28405,United States,North America,39.745947,-75.546589
15264,2099862,Lorenza,Rush,Female,10/12/1937,Riverside,CA,California,92501,United States,North America,33.721999,-116.037247


In [49]:
# Change the column names to lowercase with underscores
cleaned_df.columns = ['customer_id', 'first_name', 'last_name', 'gender', 'birthday', 'city', 'state_code', 'state', 'zip_code', 'country', 'continent', 'latitude', 'longitude']

In [50]:
cleaned_df

,customer_id,first_name,last_name,gender,birthday,city,state_code,state,zip_code,country,continent,latitude,longitude
0,301,Lilly,Harding,Female,7/3/1939,Wandearah East,SA,South Australia,5523,Australia,Australia,-33.398609,138.047863
1,325,Madison,Hull,Female,9/27/1979,Mount Budd,WA,Western Australia,6522,Australia,Australia,-29.180810,115.636896
2,554,Claire,Ferres,Female,5/26/1947,Winjallok,VIC,Victoria,3380,Australia,Australia,-36.840260,143.191632
4,1042,Aidan,Pankhurst,Male,11/19/1965,Tawonga South,VIC,Victoria,3698,Australia,Australia,-36.762541,147.138785
5,1086,Hayden,Clegg,Male,1/20/1954,Templers,SA,South Australia,5371,Australia,Australia,-34.468680,138.746913
...,...,...,...,...,...,...,...,...,...,...,...,...,...
15261,2099600,Denisa,Duková,Female,3/25/1936,Houston,TX,Texas,77017,United States,North America,29.758938,-95.367697
15262,2099618,Justin,Solórzano,Male,2/16/1992,Mclean,VA,Virginia,22101,United States,North America,40.463179,-88.819661
15263,2099758,Svend,Petrussen,Male,11/9/1937,Wilmington,NC,North Carolina,28405,United States,North America,39.745947,-75.546589
15264,2099862,Lorenza,Rush,Female,10/12/1937,Riverside,CA,California,92501,United States,North America,33.721999,-116.037247


In [108]:
# Convert the 'birthday' column to datetime
cleaned_df['birthday'] = pd.to_datetime(cleaned_df['birthday'], format='%m/%d/%Y')

In [109]:
# Format the datetime objects to '%Y-%m-%d'
cleaned_df['birthday'] = cleaned_df['birthday'].dt.strftime('%Y-%m-%d')

In [112]:
cleaned_df['birthday'] = pd.to_datetime(cleaned_df['birthday'])

In [110]:
cleaned_df

,customer_id,first_name,last_name,gender,birthday,city,state_code,state,zip_code,country,continent,latitude,longitude
0,301,Lilly,Harding,Female,1939-07-03,Wandearah East,SA,South Australia,5523,Australia,Australia,-33.398609,138.047863
1,325,Madison,Hull,Female,1979-09-27,Mount Budd,WA,Western Australia,6522,Australia,Australia,-29.180810,115.636896
2,554,Claire,Ferres,Female,1947-05-26,Winjallok,VIC,Victoria,3380,Australia,Australia,-36.840260,143.191632
4,1042,Aidan,Pankhurst,Male,1965-11-19,Tawonga South,VIC,Victoria,3698,Australia,Australia,-36.762541,147.138785
5,1086,Hayden,Clegg,Male,1954-01-20,Templers,SA,South Australia,5371,Australia,Australia,-34.468680,138.746913
...,...,...,...,...,...,...,...,...,...,...,...,...,...
15261,2099600,Denisa,Duková,Female,1936-03-25,Houston,TX,Texas,77017,United States,North America,29.758938,-95.367697
15262,2099618,Justin,Solórzano,Male,1992-02-16,Mclean,VA,Virginia,22101,United States,North America,40.463179,-88.819661
15263,2099758,Svend,Petrussen,Male,1937-11-09,Wilmington,NC,North Carolina,28405,United States,North America,39.745947,-75.546589
15264,2099862,Lorenza,Rush,Female,1937-10-12,Riverside,CA,California,92501,United States,North America,33.721999,-116.037247


In [113]:
cleaned_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 14065 entries, 0 to 15265
Data columns (total 13 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   customer_id  14065 non-null  int64         
 1   first_name   14065 non-null  object        
 2   last_name    14065 non-null  object        
 3   gender       14065 non-null  object        
 4   birthday     14065 non-null  datetime64[ns]
 5   city         14065 non-null  object        
 6   state_code   14065 non-null  object        
 7   state        14065 non-null  object        
 8   zip_code     14065 non-null  object        
 9   country      14065 non-null  object        
 10  continent    14065 non-null  object        
 11  latitude     14065 non-null  float64       
 12  longitude    14065 non-null  float64       
dtypes: datetime64[ns](1), float64(2), int64(1), object(9)
memory usage: 1.5+ MB


In [115]:
# Save the updated dataframe to a new CSV file
cleaned_df.to_csv('Output/customers.csv', index=False)

In [116]:
product_path = 'Resources/Products.csv'
sales_path = 'Resources/Sales.csv'
store_path = 'Resources/Stores.csv'

In [117]:
product_df = pd.read_csv(product_path)
sales_df = pd.read_csv(sales_path)
store_df = pd.read_csv(store_path)

# Data cleaning
## Product table
- Change the column names to lowercase with underscores and reorder
- Remove the dollar sign, commas, and trim spaces, then convert to `float`
- Drop rows with any `null` values
- Change the data type of 'unit_price_usd' to `float`
- Save the updated dataframe to a new CSV file in Output folder

In [56]:
# Display `product_df` datafrme
product_df.head()

,ProductKey,Product Name,Brand,Color,Unit Cost USD,Unit Price USD,SubcategoryKey,Subcategory,CategoryKey,Category
0,1,Contoso 512MB MP3 Player E51 Silver,Contoso,Silver,$6.62,$12.99,101,MP4&MP3,1,Audio
1,2,Contoso 512MB MP3 Player E51 Blue,Contoso,Blue,$6.62,$12.99,101,MP4&MP3,1,Audio
2,3,Contoso 1G MP3 Player E100 White,Contoso,White,$7.40,$14.52,101,MP4&MP3,1,Audio
3,4,Contoso 2G MP3 Player E200 Silver,Contoso,Silver,$11.00,$21.57,101,MP4&MP3,1,Audio
4,5,Contoso 2G MP3 Player E200 Red,Contoso,Red,$11.00,$21.57,101,MP4&MP3,1,Audio


In [63]:
# Change the column names to lowercase with underscores
product_df.columns = ['product_id', 'product_name', 'brand', 'color', 'unit_cost_usd', 'unit_pric_usd', 'subcategory_id', 'subcategory', 'category_id', 'category']

In [64]:
# Display `product_df` datafrme
product_df.head()

,product_id,product_name,brand,color,unit_cost_usd,unit_pric_usd,subcategory_id,subcategory,category_id,category
0,1,Contoso 512MB MP3 Player E51 Silver,Contoso,Silver,6.62,$12.99,101,MP4&MP3,1,Audio
1,2,Contoso 512MB MP3 Player E51 Blue,Contoso,Blue,6.62,$12.99,101,MP4&MP3,1,Audio
2,3,Contoso 1G MP3 Player E100 White,Contoso,White,7.40,$14.52,101,MP4&MP3,1,Audio
3,4,Contoso 2G MP3 Player E200 Silver,Contoso,Silver,11.00,$21.57,101,MP4&MP3,1,Audio
4,5,Contoso 2G MP3 Player E200 Red,Contoso,Red,11.00,$21.57,101,MP4&MP3,1,Audio


In [67]:
# Remove the dollar sign, commas, and trim spaces, then convert to float

product_df['unit_pric_usd'] = product_df['unit_pric_usd'].str.replace('$', '').str.replace(',', '').str.strip().astype(float)

In [70]:
# Correct the name of the column
product_df['unit_price_usd'] = product_df['unit_pric_usd']

In [73]:
# Drop the original 'unit_pric_usd' column
product_df.drop(columns=['unit_pric_usd'], inplace=True)

In [75]:
# Reorder the corrected columns
product_df.columns = ['product_id', 'product_name', 'brand', 'color', 'unit_cost_usd', 'unit_price_usd', 'subcategory_id', 'subcategory', 'category_id', 'category']

In [77]:
# Count `cleaned_product_df` dataframe
len(product_df)

2517

In [78]:
# Drop rows with any null values
cleaned_product_df = product_df.dropna()

In [82]:
# Change the data type of 'unit_price_usd' to float
cleaned_product_df['unit_price_usd']= cleaned_product_df['unit_price_usd'].astype(float)

In [79]:
# Count `cleaned_product_df` dataframe
len(cleaned_product_df)

2517

In [83]:
# Display `cleaned_product_df` datafrme
cleaned_product_df.head()

,product_id,product_name,brand,color,unit_cost_usd,unit_price_usd,subcategory_id,subcategory,category_id,category
0,1,Contoso 512MB MP3 Player E51 Silver,Contoso,Silver,6.62,101.0,MP4&MP3,1,Audio,12.99
1,2,Contoso 512MB MP3 Player E51 Blue,Contoso,Blue,6.62,101.0,MP4&MP3,1,Audio,12.99
2,3,Contoso 1G MP3 Player E100 White,Contoso,White,7.40,101.0,MP4&MP3,1,Audio,14.52
3,4,Contoso 2G MP3 Player E200 Silver,Contoso,Silver,11.00,101.0,MP4&MP3,1,Audio,21.57
4,5,Contoso 2G MP3 Player E200 Red,Contoso,Red,11.00,101.0,MP4&MP3,1,Audio,21.57


In [84]:
# Get information about dataframe
cleaned_product_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2517 entries, 0 to 2516
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   product_id      2517 non-null   int64  
 1   product_name    2517 non-null   object 
 2   brand           2517 non-null   object 
 3   color           2517 non-null   object 
 4   unit_cost_usd   2517 non-null   float64
 5   unit_price_usd  2517 non-null   float64
 6   subcategory_id  2517 non-null   object 
 7   subcategory     2517 non-null   int64  
 8   category_id     2517 non-null   object 
 9   category        2517 non-null   float64
dtypes: float64(3), int64(2), object(5)
memory usage: 196.8+ KB


In [85]:
# Save the updated dataframe to a new CSV file
cleaned_product_df.to_csv('Output/products.csv', index=False)

# Data cleaning
## Store table
- Change the column names to lowercase with underscores and reorder
- Fill null values with a default value of zero
- Convert the 'open_date' column to `datetime` and to '%Y-%m-%d' format
- Save the updated dataframe to a new CSV file in Output folder

In [21]:
# Display `store_df` datafrme
store_df

,store_id,country,state,square_meters,open_date
0,1,Australia,Australian Capital Territory,595.0,1/1/2008
1,2,Australia,Northern Territory,665.0,1/12/2008
2,3,Australia,South Australia,2000.0,1/7/2012
3,4,Australia,Tasmania,2000.0,1/1/2010
4,5,Australia,Victoria,2000.0,12/9/2015
...,...,...,...,...,...
62,63,United States,Utah,2000.0,3/6/2008
63,64,United States,Washington DC,1330.0,1/1/2010
64,65,United States,West Virginia,1785.0,1/1/2012
65,66,United States,Wyoming,840.0,1/1/2014


In [22]:
# Change the column names to lowercase with underscores
store_df.columns = ['store_id', 'country', 'state', 'square_meters', 'open_date']

In [23]:
# Display `store_df` datafrme
store_df

,store_id,country,state,square_meters,open_date
0,1,Australia,Australian Capital Territory,595.0,1/1/2008
1,2,Australia,Northern Territory,665.0,1/12/2008
2,3,Australia,South Australia,2000.0,1/7/2012
3,4,Australia,Tasmania,2000.0,1/1/2010
4,5,Australia,Victoria,2000.0,12/9/2015
...,...,...,...,...,...
62,63,United States,Utah,2000.0,3/6/2008
63,64,United States,Washington DC,1330.0,1/1/2010
64,65,United States,West Virginia,1785.0,1/1/2012
65,66,United States,Wyoming,840.0,1/1/2014


In [18]:
# Fill null values with a default value of zero
store_df.fillna(0, inplace=True)

In [24]:
# Display `store_df` datafrme
store_df

,store_id,country,state,square_meters,open_date
0,1,Australia,Australian Capital Territory,595.0,1/1/2008
1,2,Australia,Northern Territory,665.0,1/12/2008
2,3,Australia,South Australia,2000.0,1/7/2012
3,4,Australia,Tasmania,2000.0,1/1/2010
4,5,Australia,Victoria,2000.0,12/9/2015
...,...,...,...,...,...
62,63,United States,Utah,2000.0,3/6/2008
63,64,United States,Washington DC,1330.0,1/1/2010
64,65,United States,West Virginia,1785.0,1/1/2012
65,66,United States,Wyoming,840.0,1/1/2014


In [26]:
store_df['open_date'] = pd.to_datetime(store_df['open_date'], format='%m/%d/%Y')

In [27]:
# If the dates are in a specific format, specify the format
store_df['open_date'] = store_df['open_date'].dt.strftime('%Y-%m-%d')

In [28]:
# Convert the 'open-date' column to datetime
store_df['open_date'] = pd.to_datetime(store_df['open_date'])

In [29]:
# Get information about dataframe
store_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 67 entries, 0 to 66
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   store_id       67 non-null     int64         
 1   country        67 non-null     object        
 2   state          67 non-null     object        
 3   square_meters  67 non-null     float64       
 4   open_date      67 non-null     datetime64[ns]
dtypes: datetime64[ns](1), float64(1), int64(1), object(2)
memory usage: 2.7+ KB


In [30]:
store_df

,store_id,country,state,square_meters,open_date
0,1,Australia,Australian Capital Territory,595.0,2008-01-01
1,2,Australia,Northern Territory,665.0,2008-01-12
2,3,Australia,South Australia,2000.0,2012-01-07
3,4,Australia,Tasmania,2000.0,2010-01-01
4,5,Australia,Victoria,2000.0,2015-12-09
...,...,...,...,...,...
62,63,United States,Utah,2000.0,2008-03-06
63,64,United States,Washington DC,1330.0,2010-01-01
64,65,United States,West Virginia,1785.0,2012-01-01
65,66,United States,Wyoming,840.0,2014-01-01


In [141]:
# Save the updated dataframe to a new CSV file
store_df.to_csv('Output/stores.csv', index=False)

# Data cleaning
## Sales table
- Change the column names to lowercase with underscores and reorder
- Convert the 'order_date' and 'delivery_date' columns to `datetime` and to '%Y-%m-%d' format
- Save the updated dataframe to a new CSV file in Output folder

In [105]:
sales_df

,order_id,line_item,order_date,delivery_date,customer_id,store_id,product_id,quantity,currency_code
0,366000,1,2016-01-01,NaT,265598,10,1304,1,CAD
1,366001,1,2016-01-01,2016-01-13,1269051,0,1048,2,USD
2,366001,2,2016-01-01,2016-01-13,1269051,0,2007,1,USD
3,366002,1,2016-01-01,2016-01-12,266019,0,1106,7,CAD
4,366002,2,2016-01-01,2016-01-12,266019,0,373,1,CAD
...,...,...,...,...,...,...,...,...,...
62879,2243030,1,2021-02-20,NaT,1216913,43,632,3,USD
62880,2243031,1,2021-02-20,2021-02-24,511229,0,98,4,EUR
62881,2243032,1,2021-02-20,2021-02-23,331277,0,1613,2,CAD
62882,2243032,2,2021-02-20,2021-02-23,331277,0,1717,2,CAD


In [106]:
# Change the column names to lowercase with underscores
sales_df.columns = ['order_number', 'line_item', 'order_date', 'delivery_date', 'customer_id', 'store_id', 'product_id', 'quantity', 'currency_code']

In [107]:
sales_df

,order_number,line_item,order_date,delivery_date,customer_id,store_id,product_id,quantity,currency_code
0,366000,1,2016-01-01,NaT,265598,10,1304,1,CAD
1,366001,1,2016-01-01,2016-01-13,1269051,0,1048,2,USD
2,366001,2,2016-01-01,2016-01-13,1269051,0,2007,1,USD
3,366002,1,2016-01-01,2016-01-12,266019,0,1106,7,CAD
4,366002,2,2016-01-01,2016-01-12,266019,0,373,1,CAD
...,...,...,...,...,...,...,...,...,...
62879,2243030,1,2021-02-20,NaT,1216913,43,632,3,USD
62880,2243031,1,2021-02-20,2021-02-24,511229,0,98,4,EUR
62881,2243032,1,2021-02-20,2021-02-23,331277,0,1613,2,CAD
62882,2243032,2,2021-02-20,2021-02-23,331277,0,1717,2,CAD


In [33]:
# If the dates are in a specific format, specify the format
sales_df['order_date'] = pd.to_datetime(sales_df['order_date'], format='%m/%d/%Y')
sales_df['delivery_date'] = pd.to_datetime(sales_df['delivery_date'], format='%m/%d/%Y')

In [34]:
# If the dates are in a specific format, specify the format
sales_df['order_date'] = sales_df['order_date'].dt.strftime('%Y-%m-%d')
sales_df['delivery_date'] = sales_df['delivery_date'].dt.strftime('%Y-%m-%d')

In [35]:
# Convert the 'order_date' and 'delivery_date' columns to datetime
sales_df['order_date'] = pd.to_datetime(sales_df['order_date'])
sales_df['delivery_date'] = pd.to_datetime(sales_df['delivery_date'])

In [108]:
sales_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 62884 entries, 0 to 62883
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   order_number   62884 non-null  int64         
 1   line_item      62884 non-null  int64         
 2   order_date     62884 non-null  datetime64[ns]
 3   delivery_date  13165 non-null  datetime64[ns]
 4   customer_id    62884 non-null  int64         
 5   store_id       62884 non-null  int64         
 6   product_id     62884 non-null  int64         
 7   quantity       62884 non-null  int64         
 8   currency_code  62884 non-null  object        
dtypes: datetime64[ns](2), int64(6), object(1)
memory usage: 4.3+ MB


In [109]:
# Save the updated dataframe to a new CSV file
sales_df.to_csv('Output/sales.csv', index=False)

# Database creation
A short explanation of how the SQLite database is created and populated with data from CSV files:
- Database Creation: The script connects to an SQLite database named sales_dashboard.sqlite. If the database does not exist, it is created..
- Table Definitions: Creates customers, products, sales, stores with specified columns and data types.
- CSV Data Loading: Reads CSV files, converts necessary columns to date and float data types.
- Data Insertion: Inserts the data into corresponding tables in the database.
- Commit and Close: Commits changes and closes the database connection.

In [65]:
# Function to create tables with constraints and data types
def create_tables(conn):
    try:
        cursor = conn.cursor()
        
        # Create customers table
        cursor.execute('''
            CREATE TABLE IF NOT EXISTS customers (
                customer_id INTEGER PRIMARY KEY,
                first_name TEXT NOT NULL,
                last_name TEXT NOT NULL,
                gender TEXT,
                birthday DATE,
                city TEXT,
                state_code TEXT,
                state TEXT,
                zip_code TEXT,
                country TEXT,
                continent TEXT,
                latitude REAL,
                longitude REAL
            )
        ''')
        
        # Create products table
        cursor.execute('''
            CREATE TABLE IF NOT EXISTS products (
                product_id INTEGER PRIMARY KEY,
                product_name TEXT NOT NULL,
                brand TEXT,
                color TEXT,
                unit_cost_usd REAL,
                unit_price_usd REAL,
                subcategory_id INTEGER,
                subcategory TEXT,
                category_id INTEGER,
                category TEXT
            )
        ''')
        
        # Create sales table
        cursor.execute('''
            CREATE TABLE IF NOT EXISTS sales (
                order_id INTEGER PRIMARY KEY,
                line_item INTEGER,
                order_date DATE,
                delivery_date DATE,
                customer_id INTEGER,
                store_id INTEGER,
                product_id INTEGER,
                quantity INTEGER,
                currency_code TEXT,
                FOREIGN KEY(customer_id) REFERENCES customers(customer_id),
                FOREIGN KEY(product_id) REFERENCES products(product_id),
                FOREIGN KEY(store_id) REFERENCES stores(store_id)
            )
        ''')
        
        # Create stores table
        cursor.execute('''
            CREATE TABLE IF NOT EXISTS stores (
                store_id INTEGER PRIMARY KEY,
                country TEXT,
                state TEXT,
                square_meters REAL,
                open_date DATE
            )
        ''')
        
        print("Tables created successfully.")
    except Exception as e:
        print(f"Error creating tables: {e}")

In [75]:
# Function to load data from CSV files into tables with data type conversions
def load_data_from_csv(conn, csv_file, table_name, date_columns=None, float_columns=None):
    try:
        df = pd.read_csv(csv_file, encoding='latin1')
        
        # Convert specified columns to date data type
        if date_columns:
            for date_column in date_columns:
                df[date_column] = pd.to_datetime(df[date_column], errors='coerce').dt.strftime('%Y-%m-%d')
        
        # Convert specified columns to float data type
        if float_columns:
            for float_column in float_columns:
                df[float_column] = df[float_column].astype(float)
        
        df.to_sql(table_name, conn, if_exists='replace', index=False)
        print(f"Data loaded into table {table_name} successfully.")
    except Exception as e:
        print(f"Error loading data into table {table_name}: {e}")

In [76]:
# Connect to SQLite database (will create it if it doesn't exist)
conn = sqlite3.connect('sales_dashboard.sqlite')


In [78]:
# Create tables
create_tables(conn)

Tables created successfully.


In [79]:
# Base directory where the CSV files are located
base_dir = 'Output'


In [80]:
# Verify current working directory
print(f"Current Working Directory: {os.getcwd()}")

Current Working Directory: c:\Users\Gedle G\OneDrive\Documents\GitHub\Project3_Team7


In [81]:
# Verify files in Resources directory
try:
    print(f"Files in Output: {os.listdir(base_dir)}")
except FileNotFoundError:
    print(f"Directory {base_dir} not found. Please check the path.")


Files in Output: ['customers.csv', 'products.csv', 'sales.csv', 'stores.csv']


In [82]:
# List of CSV files and corresponding table names
csv_files = {
    'customers.csv': ('customers', ['birthday'], ['latitude', 'longitude']),
    'products.csv': ('products', [], ['unit_cost_usd', 'unit_price_usd']),
    'sales.csv': ('sales', ['order_date', 'delivery_date'], []),
    'stores.csv': ('stores', ['open_date'], ['square_meters'])
}

In [83]:
# Load data from CSV files into tables with data type conversions
for csv_file, (table_name, date_columns, float_columns) in csv_files.items():
    file_path = os.path.join(base_dir, csv_file)
    if os.path.exists(file_path):
        load_data_from_csv(conn, file_path, table_name, date_columns, float_columns)
    else:
        print(f"File {file_path} not found. Please check the path.")

Data loaded into table customers successfully.
Data loaded into table products successfully.
Data loaded into table sales successfully.
Data loaded into table stores successfully.


In [84]:
# Commit changes and close connection
conn.commit()
conn.close()

In [119]:
# Function to create a table with specified schema
def create_table_with_schema(conn, table_name, schema):
    try:
        conn.execute(f"CREATE TABLE IF NOT EXISTS {table_name} ({schema})")
        print(f"Table {table_name} created successfully.")
    except Exception as e:
        print(f"Error creating table {table_name}: {e}")

In [120]:
# Function to load data from CSV files into tables with data type conversions
def load_data_from_csv(conn, csv_file, table_name, date_columns=None, float_columns=None):
    try:
        df = pd.read_csv(csv_file, encoding='latin1')
        
        # Convert specified columns to date data type and format them
        if date_columns:
            for date_column in date_columns:
                df[date_column] = pd.to_datetime(df[date_column], errors='coerce').dt.strftime('%Y-%m-%d')
        
        # Convert specified columns to float data type
        if float_columns:
            for float_column in float_columns:
                df[float_column] = df[float_column].astype(float).round(2)  # Round to two decimal places
        
        # Insert the data into the table
        df.to_sql(table_name, conn, if_exists='append', index=False)
        print(f"Data loaded into table {table_name} successfully.")
    except Exception as e:
        print(f"Error loading data into table {table_name}: {e}")

In [121]:
# Connect to SQLite database (will create it if it doesn't exist)
conn = sqlite3.connect('sales_dashboard.sqlite')


In [122]:
# Table schemas
schemas = {
    'customers': """
        customer_id INTEGER PRIMARY KEY,
        first_name TEXT,
        last_name TEXT,
        gender TEXT,
        birthday DATE,
        city TEXT,
        state_code TEXT,
        state TEXT,
        zip_code TEXT,
        country TEXT,
        continent TEXT,
        latitude FLOAT,
        longitude FLOAT
    """,
    'products': """
        product_id INTEGER PRIMARY KEY,
        product_name TEXT,
        brand TEXT,
        color TEXT,
        unit_cost_usd FLOAT,
        unit_price_usd FLOAT,
        subcategory_id INTEGER,
        subcategory TEXT,
        category_id INTEGER,
        category TEXT
    """,
    'sales': """
        order_number INTEGER ,
        line_item INTEGER,
        order_date DATE,
        delivery_date DATE,
        customer_id INTEGER,
        store_id INTEGER,
        product_id INTEGER,
        quantity INTEGER,
        currency_code TEXT,
        FOREIGN KEY(customer_id) REFERENCES customers(customer_id),
        FOREIGN KEY(product_id) REFERENCES products(product_id),
        FOREIGN KEY(store_id) REFERENCES stores(store_id)
    """,
    'stores': """
        store_id INTEGER PRIMARY KEY,
        country TEXT,
        state TEXT,
        square_meters FLOAT,
        open_date DATE
    """
}

In [123]:
# Create tables with specified schemas
for table_name, schema in schemas.items():
    create_table_with_schema(conn, table_name, schema)


Table customers created successfully.
Table products created successfully.
Table sales created successfully.
Table stores created successfully.


In [124]:
# Define the CSV files and their corresponding table names, date columns, and float columns
csv_files = {
    'customers.csv': {
        'table_name': 'customers',
        'date_columns': ['birthday'],
        'float_columns': ['latitude', 'longitude']
    },
    'products.csv': {
        'table_name': 'products',
        'date_columns': [],
        'float_columns': ['unit_cost_usd', 'unit_price_usd']
    },
    'sales.csv': {
        'table_name': 'sales',
        'date_columns': ['order_date', 'delivery_date'],
        'float_columns': []
    },
    'stores.csv': {
        'table_name': 'stores',
        'date_columns': ['open_date'],
        'float_columns': ['square_meters']
    }
}


In [125]:
# Load data from CSV files into tables with data type conversions
for csv_file, attributes in csv_files.items():
    file_path = os.path.join('Output', csv_file)
    if os.path.exists(file_path):
        load_data_from_csv(conn, file_path, attributes['table_name'], attributes['date_columns'], attributes['float_columns'])
    else:
        print(f"File {file_path} not found. Please check the path.")

Data loaded into table customers successfully.
Data loaded into table products successfully.
Data loaded into table sales successfully.
Data loaded into table stores successfully.


In [126]:
# Commit changes and close connection
conn.commit()
conn.close()